In [11]:
from imports import *
from preprocessing_util import get_column_groups, get_transformers
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, precision_score, f1_score, make_scorer
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from sklearn.metrics import make_scorer, average_precision_score
from sklearn.preprocessing import OneHotEncoder

In [3]:
query = "select * from train_data"
train_data = sql_connect(query)
X_train = train_data.drop(columns=['TARGET'])
y_train = train_data['TARGET']
del train_data
X_train.head()

Connection to SQL Server established successfully.
Connection closed.


,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AVERAGE_EXTERNAL_RATING,TOT_PREV_APP,PREV_APPROVED_CNT,PREV_CANCELLED_CNT,PREV_REFUSED_CNT,PREV_UNUSED_CNT,ISMISSING_BUREAU_INFO
0,Cash loans,M,0,1,1,126000.0,545040.0,17244.0,450000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.010966,-9470,-1114,-9301.0,-2142.0,-1,1,1,0,1,1,0,Drivers,3,2,2,TUESDAY,15,0,0,0,0,1,1,Business Entity Type 3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,1.0,0.5739,0,0,0,0,0,0
1,Cash loans,F,0,1,0,135000.0,269550.0,18760.5,225000.0,Unaccompanied,Pensioner,Secondary / secondary special,Married,House / apartment,0.035792,-24367,365243,-13226.0,-4579.0,-1,1,0,0,1,1,0,Retired,2,2,2,WEDNESDAY,15,0,0,0,0,0,0,NA,0,0,0,0,-1052,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.7012,0,0,0,0,0,1
2,Cash loans,M,0,0,0,270000.0,497520.0,33246.0,450000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.008474,-10739,-353,-1418.0,-1843.0,-1,1,1,1,1,0,0,Laborers,2,2,2,SUNDAY,11,1,1,0,1,1,0,Business Entity Type 2,0,0,0,0,-251,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.0,0.0,0.0,1.0,0.3572,0,0,0,0,0,0
3,Revolving loans,F,0,1,0,112500.0,315000.0,15750.0,315000.0,Family,Pensioner,Secondary / secondary special,Widow,House / apartment,0.035792,-21202,365243,-15132.0,-3152.0,-1,1,0,0,1,1,0,Retired,1,2,2,SATURDAY,11,0,0,0,0,0,0,NA,0,0,0,0,-551,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,2.0,8.0,0.5516,0,0,0,0,0,0
4,Revolving loans,F,1,1,2,225000.0,450000.0,22500.0,450000.0,Unaccompanied,Working,Higher education,Married,House / apartment,0.028663,-12341,-4904,-6217.0,-4151.0,24,1,1,0,1,0,0,Accountants,4,2,2,WEDNESDAY,11,0,0,0,1,0,1,Business Entity Type 2,0,0,0,0,-292,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.4927,0,0,0,0,0,0


In [18]:
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
rf_baseline = RandomForestClassifier(
    class_weight='balanced', 
    random_state=42,
    n_jobs=-1
)

scores = cross_validate(
    rf_baseline,
    X_train,
    y_train,
    cv=cv_strategy,
    scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'],
    return_train_score=False
)

print("Baseline Random Forest CV Results:")
for metric, values in scores.items():
    print(f"{metric}: {np.mean(values):.4f}")


In [ ]:
scoring = {
    'recall': make_scorer(recall_score, average='weighted', zero_division=0),
    'precision': make_scorer(precision_score, average='weighted', zero_division=0),
    'f1': make_scorer(f1_score, average='weighted', zero_division=0)
    #'pr_auc': make_scorer(average_precision_score, needs_proba=True, )
}

In [12]:
categorical_cols = [col for col in X_train.columns if X_train[col].dtype == 'object']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'
)

In [19]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])



# 4. Cross-validation
results = cross_validate(pipeline, X_train, y_train, cv=cv_strategy, scoring=scoring)

# 5. Print average metrics
print("Baseline Random Forest - Cross-Validation Metrics:")
for metric in scoring:
    print(f"{metric.upper():<8}: {np.mean(results[f'test_{metric}']):.3f}")

c:\Users\adars\Documents\Ireland_Project\Loan_approval_ml\loan_approval_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


KeyboardInterrupt: 